In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Read Inside Airbnb data") \
    .getOrCreate()

24/11/23 22:51:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
listings = spark.read.csv("../data/listings.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",", 
    quote='"',
    escape='"', 
    multiLine=True,
    mode="PERMISSIVE" 
)

In [4]:
listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: int

In [5]:
# Get a non-null picture URL for any property ("picture_url" field)
# Select any non-null picture URL
listings.filter(
    listings.picture_url.isNotNull()
) \
.select('picture_url') \
.limit(1) \
.show(truncate=False)

+------------------------------------------------------------------------------------------------------+
|picture_url                                                                                           |
+------------------------------------------------------------------------------------------------------+
|https://a0.muscache.com/pictures/miso/Hosting-13913/original/d755aa6d-cebb-4464-80be-2722c921e8d5.jpeg|
+------------------------------------------------------------------------------------------------------+



In [6]:
# Get number of properties that get more than 10 reviews per month
listings.filter(
  listings.reviews_per_month > 10
) \
.count()

66

In [7]:
# Get a property that has more bathrooms than bedrooms
listings.filter(
    (listings.bathrooms > listings.bedrooms)
) \
.select('name', 'bathrooms', 'bedrooms') \
.show(10, truncate=False)

+--------------------------------------------------+---------+--------+
|name                                              |bathrooms|bedrooms|
+--------------------------------------------------+---------+--------+
|Battersea live/work artist house, garden & parking|1.5      |1       |
|West London-W7, Hanwell(Area Ealing) Room (Female)|1.5      |1       |
|West London,loft ensuite, 5min2tube               |1.5      |1       |
|Large Bedroom with EnSuite Bathroom               |1.5      |1       |
|Cosy Double studio in Zone 2 Hammersmith (1)      |1.5      |1       |
|Shoreditch Loft                                   |1.5      |1       |
|Five minute walk to South Bank                    |1.5      |1       |
|Designer room Park Views 4 mins zone 1 station    |1.5      |1       |
|Cosy Double room London, River View               |1.5      |1       |
|Maisonette in Central London Zone 1               |1.5      |1       |
+--------------------------------------------------+---------+--

In [12]:
# Get 10 properties where the price is greater than 3,000. Collect it as a list
from pyspark.sql.functions import regexp_replace

listings_with_price = listings \
  .withColumn('price_numeric', regexp_replace('price', '[$,]', '').cast('float'))

res = listings_with_price.filter(
    (listings_with_price.price_numeric > 3000)
) \
.select('name', 'price') \
.collect()

res

[Row(name='HUGE DOUBLE ROOM WITH MEZZANINE', price='$5,000.00'),
 Row(name='Bright ,Modern, 12m to Bond street.', price='$5,500.00'),
 Row(name='Bright airy large Victorian house', price='$5,000.00'),
 Row(name='Spacious Double Room in E14', price='$4,000.00'),
 Row(name='Room in a cosy flat. Central, clean', price='$8,000.00'),
 Row(name='3 Bed Flat in South Hampstead with Large Garden!', price='$25,000.00'),
 Row(name='Spacious Private Ground Floor Room', price='$7,693.00'),
 Row(name='No Longer Available', price='$53,588.00'),
 Row(name='Very nice double room in the heart of Soho', price='$5,100.00'),
 Row(name='SHORT WALK TO LONDON EYE (SWITZ)', price='$5,000.00'),
 Row(name='Knightsbridge Penthouse', price='$8,895.00'),
 Row(name='Double Room with King Size Bed near Tower Bridge', price='$3,670.00'),
 Row(name='London | Modern flat for travelers and families', price='$3,500.00'),
 Row(name='Luxury 5-star Flat with Art & Tech', price='$6,276.00'),
 Row(name='The Loft Queens Park', 

In [13]:
# Get a list of properties with the following characteristics:
# * price < 150
# * more than 20 reviews
# * review_scores_rating > 4.5

listings_with_price.filter(
    (listings_with_price.price_numeric < 150) &
    (listings_with_price.number_of_reviews > 20) &
    (listings_with_price.review_scores_rating > 4.5)
  ) \
.select('name', 'price_numeric', 'number_of_reviews', 'review_scores_rating') \
.show(truncate=False)

+--------------------------------------------------+-------------+-----------------+--------------------+
|name                                              |price_numeric|number_of_reviews|review_scores_rating|
+--------------------------------------------------+-------------+-----------------+--------------------+
|Holiday London DB Room Let-on going               |59.0         |44               |4.82                |
|Bright Chelsea  Apartment. Chelsea!               |120.0        |96               |4.8                 |
|Double Room (Unavailable for check in 31Dec-1Jan) |40.0         |38               |4.89                |
|A stylish Victorian home in West London           |131.0        |91               |4.85                |
|I Bedroom flat Tower of London                    |145.0        |248              |4.9                 |
|You are GUARANTEED to love this                   |82.0         |691              |4.86                |
|SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT     

In [16]:
from pyspark.sql.functions import max

# Get the highest price in the dataset
listings_with_price.select(max('price_numeric')).show()


+------------------+
|max(price_numeric)|
+------------------+
|           80000.0|
+------------------+



In [22]:
# Get the name and a price of property with the highest number of reviews per month
from pyspark.sql.functions import length

# max_desc_len = listings_with_price.select(max(length('description'))).collect()[0]

listings_with_price.filter(listings_with_price.price_numeric == 80000.0).select('name', 'price').show()


+--------------------+----------+
|                name|     price|
+--------------------+----------+
|Room In Zone 1 (TOB)|$80,000.00|
|Close To London B...|$80,000.00|
+--------------------+----------+



In [23]:
# Get the number of hosts in the dataset
listings.select('host_name').distinct().count()

16379